In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d

import pickle
import os
import torch

import pandas as pd

from scipy.optimize import leastsq
import plotly.io as pio
pio.renderers.default = 'notebook+vscode'
from sklearn.cluster import KMeans

#import sys
#sys.path.insert(0, '/Users/danielverschueren/Documents/Sonalis/code/registration/registration/')
#sys.path.insert(0, '/Users/danielverschueren/Documents/Sonalis/code/registration/')

from point_cloud_torch import affinePC_torch, apply_affine_torch, apply_affine_inverse_torch
from point_cloud import plot_PCs, affinePC

# the fit function in affinePC calls on minimize_parallel from optimparallel
# make sure that package is installed

def pgy2npy(pgy_file):
    """
    pgy file to npy
    """
    with open(pgy_file) as f:
        lines = f.readlines()
    lines = lines[1:] # remove header
    np_data = np.zeros((len(lines), 3), np.float64)
    for i, line in enumerate(lines):
        l = []
        for t in line.split():
            try:
                l.append(float(t))
            except ValueError:
                pass
        np_data[i] = np.asarray(l[1:])
    return np_data

plot_using = 'plotly_light'

%load_ext autoreload
%autoreload 2

In [2]:
# Some useful color arrays
colour_white = np.array([[1, 1, 1]], dtype=np.float64).swapaxes(1, 0)
colour_red = np.array([[1, 0, 0]], dtype=np.float64).swapaxes(1, 0)
colour_black = np.array([[0, 0, 0]], dtype=np.float64).swapaxes(1, 0)
colour_blue = np.array([[0, 0, 1]], dtype=np.float64).swapaxes(1, 0)
colour_green = np.array([[0, 1, 0]], dtype=np.float64).swapaxes(1, 0)

In [ ]:
fitted_coords = np.load('coords/coords.npy')

In [ ]:
# reload nominal geom
nominal_geometry_points = pgy2npy('omega2-PointReceivers_in_mm.pgy')
nominal_geometry_points.T[[1,2]] = nominal_geometry_points.T[[2,1]] # swap axes
nominal_geometry_points.T[[1,0]] = nominal_geometry_points.T[[0,1]] # swap axes

origin = np.mean(nominal_geometry_points, axis=0)
fitted_origin = np.mean(fitted_coords, axis=0)
fitted_coords += origin - fitted_origin

dx = 5
dphi = 1
start = [0.,0.,np.pi-0.35,0,0,0]

#start = [ 0., 0., 2.77345857, 46.13457182, 141.2809497, 157.09586469]

bounds_r = [(start[3]-dx, start[3]+dx), (start[4]-dx, start[4]+dx), (start[5]-dx, start[5]+dx)]
bounds_phi = [(start[0]-dphi,start[0]+dphi),(start[1]-dphi,start[1]+dphi),(start[2]-dphi, start[2]+dphi)]
bounds = bounds_phi + bounds_r

print('start_fitting... fitted points')
M_PC = affinePC(nominal_geometry_points, fitted_coords)
M_PC.params = start
M_PC.fit(start=start, bounds=bounds, method='naive')
print(f"Affine parametes [rot_x, rot_y, rot_z, t_x, ty, t_z]: \n{M_PC.params}")
reg_fitted = M_PC.apply_aff()

# move back
nominal_geometry_points.T[[0,1]] = nominal_geometry_points.T[[1,0]] # swap axes
nominal_geometry_points.T[[2,1]] = nominal_geometry_points.T[[1,2]] # swap axes

reg_fitted.T[[0,1]] = reg_fitted.T[[1,0]] # swap axes
reg_fitted.T[[2,1]] = reg_fitted.T[[1,2]] # swap axes

fig = plot_PCs([nominal_geometry_points, reg_fitted])
fig.show()

## hand picked

In [ ]:
a = pd.read_csv('handpicked_240123_coordinates.csv')
print(a.head())
b = a[a.columns[1]].str.split()
a['x'] = np.zeros(len(a))
a['y'] = np.zeros(len(a))
a['z'] = np.zeros(len(a))

for i in range(len(a)):
    a.iloc[i,3] = b.iloc[i][2]
    a.iloc[i,4] = b.iloc[i][1]
    a.iloc[i,5] = b.iloc[i][0]

In [ ]:

hand_coords = a.iloc[:,3:].astype(float).to_numpy()
mean_x = np.mean(hand_coords[:,2])
hand_coords[:,2] = hand_coords[:,2] + mean_x

# reload nominal geom
nominal_geometry_points = pgy2npy('omega2-PointReceivers_in_mm.pgy')
nominal_geometry_points.T[[1,2]] = nominal_geometry_points.T[[2,1]] # swap axes
nominal_geometry_points.T[[1,0]] = nominal_geometry_points.T[[0,1]] # swap axes
center_nominal = nominal_geometry_points.mean(axis=0)

origin = np.mean(nominal_geometry_points, axis=0)
hand_origin = np.mean(hand_coords, axis=0)

hand_coords += origin - hand_origin

nominal_centered = nominal_geometry_points - center_nominal

# affine
start = torch.Tensor([0,0,0,0,0,0]) # it will be important to have a rough starting point for 
                      # each! the original geomtery will be very helpful 1600

start = torch.Tensor([0,0,0,0,0,0])

print(len(hand_coords))
print(len(nominal_centered))

print('start_fitting...')
N_PC = affinePC_torch(torch.Tensor(hand_coords), torch.Tensor(nominal_centered))
dphi = 1800
N_PC.fit(start=start, 
         bounds=[(0-dphi,0+dphi),(0-dphi,0+dphi),(0-dphi, 0+dphi)]+[(None,None)]*3, 
         method='naive',
         max_oper=1,
         lr=0.1)
print(f"Affine parametes [rot_x, rot_y, rot_z, t_x, t_y, t_z]: \n{N_PC.params}")
reg_nominal = N_PC.apply_aff()
params = N_PC.params

# construct open3D objects
reg_nom = o3d.utility.Vector3dVector(reg_nominal)
pcd_reg_nom = o3d.geometry.PointCloud(reg_nom)
pcd_reg_nom = pcd_reg_nom.paint_uniform_color(colour_green)
clusters_ = o3d.utility.Vector3dVector(hand_coords)
pcd_cluster_c = o3d.geometry.PointCloud(clusters_)
pcd_cluster_c = pcd_cluster_c.paint_uniform_color(colour_blue)

fig = plot_PCs([reg_nominal, hand_coords])
fig.show()